<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Main()" data-toc-modified-id="Main()-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Main()</a></span></li><li><span><a href="#源码-Part-1" data-toc-modified-id="源码-Part-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>源码 Part 1</a></span></li></ul></div>

## Main()

In [8]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

# from generator_file import *
from functions import *
import generator_file

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


`Main()`

In [ ]:


# 载入原始数据 & 计算 df_star
df, r, df_star, r_star = preprocess()

# 处理数据，得到 Lambda_star & mu_z
# lambda_star = compute_lambda_star()
# mu_z = compute mu_z()
# 解方程得到估计值
alpha_hat = solve(s_n)



# 带入估计值算 z_i
# z = compute_z()
# 解方程得到估计值
beta_hat = solve(u_n)



----

## 源码 Part 1

In [ ]:
def preprocess(a):
    '''
    a: estimate parameter
    '''
    global df, r, df_star, r_star
    
    df ,r = generator()
    
    # 乘法因子
    e_star = np.exp(np.array([df.x1, df.x2]).T @ a)
    
    # justify df^*
    df_star = df.copy()
    df_star.y = df.y * e_star
    df_star.c = df.c * e_star
    df_star.d = df.d * e_star

    # {t_ij^*(a)}
    r_star = r * e_star
    
#     return df, r, df_star, r_star
    

def compute_lambda(a):
    '''
    a: estimate parameter
    '''
    global lambda_a
    
    # N
    N = np.sum(df.m)

    # Next to sort {sl}
    sl = flatten(r_star)

    # {rl}
    compare = df_star.y.values.reshape((1, -1)) < sl.reshape((-1, 1))
    rl = np.arange(N) + 1 - compare @ df.m.values + 0.05

    # index
    index = np.sum((1 - compare).T, axis=1)

    # vector {\hat{\Lambda}_n}
    factor = 1 - 1 / rl
    factor = np.append(factor, 1)
    lambda_ = np.cumprod(factor[::-1])[::-1]

    # vector{\lambda(Y^*(a))}
    lambda_a = lambda_[[index]]


def compute_mu(a):
    global mu_z
    
    mu_z = df_star.m @ (1/lambda_a) /size
    
    
def s_n(a):
    
    preprocess(a)
    compute_lambda(a)
    compute_mu(a)
    result = [df.x1.values @ (df.m / lambda_a - mu_z).values, df.x2.values @ (df.m / lambda_a - mu_z).values]
    return np.array(result) / size

In [ ]:
size = 200
a = np.array([-1, 1]) # for test
# preprocess(a)
# mu_z(a)
s_n(a)

----

In [ ]:
a_hat = np.array([0, 0])
size = 200

for _ in range(50):
#     print(fsolve(s_n, [-1, 1]))
    a_hat = np.append(a_hat, np.array(fsolve(s_n, [-1, 1])))
a_hat = a_hat[2::]

In [ ]:
aaa = a_hat.reshape([-1, 2])
print(np.mean(aaa, axis=0))